# 🧠 智能体实践教学：构建你的第一个智能体

## 📘 教学目标
本次Notebook将引导你：
1. 了解什么是智能体（Agent）及其基本结构；
2. 掌握使用 `langchain` 和 `openai` 构建一个具备基础能力的智能体；
3. 亲手实现一个能对话、调用工具并具备记忆功能的智能体。

## 🧩 Step 1：什么是智能体？

我们先来了解一下基础概念。

In [1]:
from IPython.display import Markdown

Markdown('''
**智能体（Agent）是一个能够感知环境、做出判断并执行行动的实体。**
在大模型系统中，它通常具备：
- **语言模型（LLM）**：用于理解指令和生成回应；
- **工具（Tools）**：用于执行搜索、计算、数据库访问等具体任务；
- **记忆（Memory）**：存储上下文对话信息，支持多轮对话；
- **决策逻辑**：决定当前应调用哪个工具或如何响应。
''')


**智能体（Agent）是一个能够感知环境、做出判断并执行行动的实体。**
在大模型系统中，它通常具备：
- **语言模型（LLM）**：用于理解指令和生成回应；
- **工具（Tools）**：用于执行搜索、计算、数据库访问等具体任务；
- **记忆（Memory）**：存储上下文对话信息，支持多轮对话；
- **决策逻辑**：决定当前应调用哪个工具或如何响应。


## 🧪 Step 2：环境配置（必做）

运行以下代码，安装必要库并配置OpenAI的API Key：

In [2]:
%pip install langchain openai faiss-cpu tiktoken langchain-community --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chat_models import ChatOpenAI

# 请根据你的OpenAI账号填写以下信息
OPENAI_API_KEY = "2c2a258f6daf4ab1a7eaa1f1298f5a0d.Lajx17gMNJr4bk9F"
OPENAI_API_BASE = "https://open.bigmodel.cn/api/paas/v4/"  # 如果你使用的是代理或自定义服务商，可以修改此URL
OPENAI_MODEL = "glm-4-air" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE

llm = ChatOpenAI(
    model_name=OPENAI_MODEL,
    temperature=0.9,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE
)

C:\Users\pc\AppData\Local\Temp\ipykernel_40140\4228877551.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## 🧱 Step 3：构建你的第一个智能体

### 🛠️ 第一步：定义工具（Tool）

我们将实现一个加法器工具。

In [4]:
from langchain.tools import tool

@tool
def add_numbers(input: str) -> str:
    """接收两个用逗号分隔的数字字符串，如 '3,4' 返回 '7'。支持容错与格式清洗。"""
    try:
        # 移除引号、换行、冒号等常见格式污染
        input = input.strip().replace("'", "").replace("\"", "").replace("\n", "").replace("Observation:", "")
        parts = [x.strip() for x in input.split(',') if x.strip()]
        if len(parts) != 2:
            return "输入格式错误：请提供两个用逗号分隔的数字，如 '3,4'"
        a, b = parts
        return str(int(a) + int(b))
    except Exception as e:
        return f"输入格式错误，请输入形如 '3,4' 的字符串：{str(e)}"
    
@tool
def factorial_tool(input: str) -> str:
    """接收一个正整数字符串，返回其阶乘值。"""
    try:
        # 清理输入格式
        input = input.strip().replace("'", "").replace("\"", "").replace("\n", "")
        
        # 检查是否为非负整数
        if not input.isdigit():
            return "输入格式错误：请输入一个非负整数。"
        
        n = int(input)
        
        # 计算阶乘
        if n < 0:
            return "输入必须是非负整数。"
        
        factorial = 1
        for i in range(1, n + 1):
            factorial *= i
        
        return str(factorial)
    except Exception as e:
        return f"计算阶乘时发生错误：{str(e)}"

### 🤖 第二步：初始化Agent

In [5]:
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

# 添加工具列表
tools = [
    Tool(
        name="AddNumbers",
        func=add_numbers,
        description="用于执行两个数字加法，例如输入 '3,4'"
    ),
        Tool(
        name="Factorial",
        func=factorial_tool,
        description="计算一个非负整数的阶乘，例如输入 '5' 返回 '120'"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\pc\AppData\Local\Temp\ipykernel_40140\2178452909.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### 🧪 第三步：运行测试
langchain中的agent遵循ReAct范式

相关阅读：ReAct: Synergizing Reasoning and Acting in Language Models

In [6]:
agent.run("请帮我计算3加7")

C:\Users\pc\AppData\Local\Temp\ipykernel_40140\339764922.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("请帮我计算3加7")




> Entering new AgentExecutor chain...
Thought: 我需要将数字3和数字7相加，得到它们的和。我可以使用AddNumbers这个工具来完成这个任务。

Action: AddNumbers
Action Input: '3,7'
Observation:
Observation: 10
Thought:Thought: 通过使用AddNumbers工具，我已经得到了3和7相加的结果，即10。

Final Answer: 10

> Finished chain.


'10'

📝 **问题思考**：你能修改输入内容让Agent报错吗？为什么会出错？

In [7]:
#agent.run("请帮我计算3.5加7")

因为AddNumbers计算为整数计算

🧠 **自主练习**：请你实现一个阶乘计算工具 `factorial_tool`，接收一个正整数字符串返回其阶乘值，如输入 '5' 应返回 '120'。要求：
- 检查输入是否为非负整数
- 对非法输入添加适当错误提示
- 将其添加至 `tools` 列表

In [8]:
agent.run("请计算5的阶乘")



> Entering new AgentExecutor chain...
To calculate the factorial of 5, I need to use the Factorial tool.
Action: Factorial
Action Input: '5'
Observation: 120
Thought:I now know the final answer
Final Answer: 120

> Finished chain.


'120'

## 🔁 Step 4：丰富你的Agent记忆能力

In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

agent_with_memory = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

C:\Users\pc\AppData\Local\Temp\ipykernel_40140\732010614.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [10]:
agent_with_memory.run("请你帮我计算12+47等于多少")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: AddNumbers
Action Input: 12,47
Observation:
Observation: 59
Thought:Thought: Do I need to use a tool? No
AI: 12+47等于59。

> Finished chain.


'12+47等于59。'

In [11]:
agent_with_memory.run("我刚才的计算结果是多少？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 你刚才的计算结果是59。

> Finished chain.


'你刚才的计算结果是59。'

In [12]:
agent_with_memory.run("在刚才的基础上再加23，结果是多少？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: AddNumbers
Action Input: 59,23
Observation:
Observation: 82
Thought:Thought: Do I need to use a tool? No
AI: 在刚才的基础上再加23，结果是82。

> Finished chain.


'在刚才的基础上再加23，结果是82。'

In [13]:
print("\n--- 当前记忆内容 ---")
for msg in memory.chat_memory.messages:
    print(f"{msg.type.upper()}: {msg.content}")


--- 当前记忆内容 ---
HUMAN: 请你帮我计算12+47等于多少
AI: 12+47等于59。
HUMAN: 我刚才的计算结果是多少？
AI: 你刚才的计算结果是59。
HUMAN: 在刚才的基础上再加23，结果是多少？
AI: 在刚才的基础上再加23，结果是82。


### 使用摘要记忆
摘要记忆通过将对话历史生成摘要，适合长对话中保持语义一致性而不被信息爆炸干扰。

In [14]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history",return_messages=True,output_key="output")

agent_summary = initialize_agent(
    tools=tools,
    llm=llm,
    memory=summary_memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\pc\AppData\Local\Temp\ipykernel_40140\1656635570.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history",return_messages=True,output_key="output")


**练习**：试试连续输入三轮关于旅游计划的问题，观察模型是否能总结用户的目标。

In [15]:
# 进行三轮模拟对话
agent_summary.run("我想去日本旅游，大概在10月中旬出发。")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 好的，去日本旅游是个不错的选择。10月中旬的天气比较舒适，适合旅游。您想了解一些关于日本旅游的信息吗，比如签证、行程推荐等？

> Finished chain.


'好的，去日本旅游是个不错的选择。10月中旬的天气比较舒适，适合旅游。您想了解一些关于日本旅游的信息吗，比如签证、行程推荐等？'

In [16]:
agent_summary.run("我更倾向于去京都和大阪，不太喜欢太过拥挤的城市。")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Understood. Kyoto and Osaka are great choices for a less crowded experience in Japan. Kyoto is known for its rich history and beautiful temples, while Osaka offers a vibrant city life with a friendly atmosphere. Would you like me to provide information on specific sights to visit in these cities or help you plan your itinerary?

> Finished chain.


'Understood. Kyoto and Osaka are great choices for a less crowded experience in Japan. Kyoto is known for its rich history and beautiful temples, while Osaka offers a vibrant city life with a friendly atmosphere. Would you like me to provide information on specific sights to visit in these cities or help you plan your itinerary?'

In [17]:
agent_summary.run("希望整个行程控制在7天之内，预算大概是1万元人民币以内。")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 您计划在7天之内以1万元人民币的预算旅行京都和大阪，这个计划是可行的。关于签证，中国公民前往日本旅游通常需要办理旅游签证。我会提供一些建议来帮助您规划行程。

1. 京都和大阪的住宿：您可以选择经济型酒店或青旅，以控制预算。
2. 交通：考虑购买日本铁路周游券（JR Pass），这样可以节省在城市间移动的费用。
3. 饮食：您可以选择在日本当地的小吃店或家庭餐馆用餐，这样的费用相对较低。
4. 景点门票：一些景点可能需要门票，您可以提前规划好想要参观的景点，并了解门票价格。

接下来，我会为您提供一个简单的行程建议：
- 第1天：抵达大阪，适应时差，轻松游览大阪城。
- 第2天：游览大阪通天阁、道顿堀等。
- 第3天：前往京都，参观金阁寺、清水寺等。
- 第4天：在京都继续游览，如岚山、二条城等。
- 第5天：前往大阪，游览环球影城日本（USJ）。
- 第6天：在大阪购物，体验当地美食。
- 第7天：返回中国。

请注意，这只是一个基本的行程建议，您可以根据自己的兴趣和需求进行调整。另外，我会为您查找相关的签证信息和具体的景点门票价格，以便您更好地规划行程。

> Finished chain.


'您计划在7天之内以1万元人民币的预算旅行京都和大阪，这个计划是可行的。关于签证，中国公民前往日本旅游通常需要办理旅游签证。我会提供一些建议来帮助您规划行程。\n\n1. 京都和大阪的住宿：您可以选择经济型酒店或青旅，以控制预算。\n2. 交通：考虑购买日本铁路周游券（JR Pass），这样可以节省在城市间移动的费用。\n3. 饮食：您可以选择在日本当地的小吃店或家庭餐馆用餐，这样的费用相对较低。\n4. 景点门票：一些景点可能需要门票，您可以提前规划好想要参观的景点，并了解门票价格。\n\n接下来，我会为您提供一个简单的行程建议：\n- 第1天：抵达大阪，适应时差，轻松游览大阪城。\n- 第2天：游览大阪通天阁、道顿堀等。\n- 第3天：前往京都，参观金阁寺、清水寺等。\n- 第4天：在京都继续游览，如岚山、二条城等。\n- 第5天：前往大阪，游览环球影城日本（USJ）。\n- 第6天：在大阪购物，体验当地美食。\n- 第7天：返回中国。\n\n请注意，这只是一个基本的行程建议，您可以根据自己的兴趣和需求进行调整。另外，我会为您查找相关的签证信息和具体的景点门票价格，以便您更好地规划行程。'

## 🎯 自主练习
请完成以下任务，提交代码与说明：

1. **集成搜索功能**，使用 `DuckDuckGoSearchRun` 工具并问“OpenAI是谁？”；
2. **尝试更换Agent类型**为 `AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION` 并观察变化；

提示：使用如下代码添加搜索工具：
```python
!pip install -U duckduckgo-search
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools.append(Tool(name="Search", func=search.run, description="执行网页搜索"))
```


In [18]:
%pip install -U duckduckgo-search

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------- -------------------------- 1.3/3.8 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 15.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 36.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationSummaryMemory

# 初始化搜索工具
search = DuckDuckGoSearchRun()
tools.append (
    Tool(
        name="Search",
        func=search.run,
        description="执行网页搜索"
    )
)
# 测试搜索功能
response1 = agent.run("OpenAI是谁？")
print("\n第一个回答：", response1)



> Entering new AgentExecutor chain...
OpenAI是一个总部位于美国的人工智能研究公司，它的目标是创建安全、有益的人工智能系统。我需要通过工具来获取更详细的信息。

Action:搜索引擎搜索
Action Input: "OpenAI是什么"
Observation:
Observation: 搜索引擎搜索 is not a valid tool, try one of [AddNumbers, Factorial].
Thought:Thought: OpenAI的相关信息并不需要通过数学工具来获取，而是需要我直接提供基于已有知识的答案。

Final Answer: OpenAI 是一家总部位于美国的人工智能研究公司，其宗旨是推动人工智能的发展，创建安全、有益的人工智能系统，并且确保人工智能的广泛应用能够造福人类。该公司由众多科技企业家和投资者共同创立，其中包括 Elon Musk 和 Y Combinator 的总裁 Sam Altman。OpenAI 的研究内容包括自然语言处理、机器学习、机器人技术等多个领域。

> Finished chain.

第一个回答： OpenAI 是一家总部位于美国的人工智能研究公司，其宗旨是推动人工智能的发展，创建安全、有益的人工智能系统，并且确保人工智能的广泛应用能够造福人类。该公司由众多科技企业家和投资者共同创立，其中包括 Elon Musk 和 Y Combinator 的总裁 Sam Altman。OpenAI 的研究内容包括自然语言处理、机器学习、机器人技术等多个领域。


In [20]:

# 初始化对话摘要记忆
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True,
    output_key="output"
)

# 更换 Agent 类型为 CHAT_CONVERSATIONAL_REACT_DESCRIPTION
agent_chat = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# 测试搜索功能
response = agent_chat.run("OpenAI是谁？")
print("\n🔍 搜索回答：")
print(response)




> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "OpenAI是什么"
}
```
Observation: OpenAI是什么 OpenAI的发展历程. 2015年由马斯克、美国创业孵化器Y Combinator总裁阿尔特曼、全球在线支付平台PayPal联合创始人彼得·蒂尔等硅谷科技大亨创立。2016年6月21日，OpenAI宣布了其主要目标，包括制造"通用"机器人和使用自然语言的聊天机器人。OpenAI研发主管伊利娅·苏特斯科娃(Ilya Sutskever ... 本文深度分析了OpenAI从创立到成为AI巨头的转变过程，探讨了其创始人Sam Altman的动机、挑战和定位。文章还介绍了OpenAI的重要产品和项目，如ChatGPT、AGI等，以及其在AI竞争中的地位和影响。 Azure OpenAI 服务为客户提供先进的语言人工智能技术，包括 OpenAI GPT-4、GPT-3、Codex、GPT-image-1（预览）、DALL-E，以及提供语音转文本和文本转语音的模型，同时确保 Azure 的安全和企业级承诺。 Azure OpenAI 与 OpenAI 共同开发 API，确保兼容性的同时能够实现二者之间的 ... 掌握OpenAI API Key的获取与管理，确保安全高效使用。本文提供详细指南，涵盖API Key的定义、作用、重要性，国际与国内获取方式，代码示例及安全存储方法，并解答常见问题，助您充分利用OpenAI技术。 OpenAI使用了多种深度学习框架，包括TensorFlow、PyTorch、Keras和Theano等。这些框架提供了丰富的功能，用于训练和部署深度学习模型。此外，OpenAI还开发了一些自己的算法和工具，如GAN（生成对抗网络）、DALL-E、GPT等，这些算法和工具在计算机视觉、自然语言处理等 ...
Thought:```json
{
    "action": "Final Answer",
    "action_input": "OpenAI是一家总部位于美国的人工智能研究公司，由埃隆·马斯克、Y Combinator总裁阿尔特曼、PayPal联合创始人彼得·蒂尔等硅谷科技大亨于

## 📌思考
- 如果要让智能体拥有“规划一周学习任务”的能力，你会添加哪些工具和模块？

首先，我认为任务管理是规划学习任务的核心。智能体应该能够访问日程管理工具，比如 Google Calendar 或 Outlook Calendar，来查看学生的空闲时间，并合理安排学习任务。比如，它可以自动将学习内容分配到学生一周的时间表上，确保每个任务都有足够的时间完成，同时避免时间冲突。

接下来，智能体可以通过像 Todoist 或 Trello 这样的任务管理工具帮助学生创建学习清单，并跟踪任务进度。这不仅可以确保学生不会遗漏重要的学习任务，还能让他们实时看到自己的学习进度和完成情况。

然而，学习任务的规划不仅仅是安排时间，还需要学生获取适当的学习资源。为了帮助学生找到所需的资料，智能体可以集成 DuckDuckGoSearchRun 或 Google Custom Search 等工具，自动搜索与学习内容相关的课程、书籍、研究文章等资源。通过这种方式，智能体不仅能为学生节省查找资料的时间，还能确保他们找到最相关、最权威的资源。

此外，智能体应该具备自适应调整学习任务的能力。学生的学习进度和效果可能会随着时间有所变化，智能体可以通过 Reinforcement Learning（强化学习）根据学生的反馈动态调整学习计划。如果学生在某个领域进展较慢，智能体可以自动延长该部分的学习时间，或根据学生的学习习惯提供个性化的学习建议。同时，智能体还可以引导学生使用 Pomodoro Technique（番茄工作法），帮助他们集中精力，提高学习效率，避免拖延。

最后，智能体应该具备定期评估和反馈功能，以帮助学生了解自己的学习情况并及时做出调整。通过像 SurveyMonkey API 或 Google Forms 这样的工具，智能体可以收集学生对学习任务的反馈，并根据这些反馈优化后续的任务规划。智能体还可以通过 Habits Tracking API 跟踪学生的学习习惯，基于习惯数据提供更个性化的建议和反馈，进一步提高学习效率。